In [1]:
!pip install spacy


In [2]:
!pip install xgboost

In [3]:
import pandas as pd
import numpy as np
import re
import joblib
import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

In [4]:
try:
    nlp = spacy.load("es_core_news_sm")
except:
    # Si no está instalado, instalarlo
    print("Instalando modelo de spaCy para español...")
    !python -m spacy download es_core_news_sm
    nlp = spacy.load("es_core_news_sm")

In [5]:
class TextTokenizer(BaseEstimator, TransformerMixin):

    def __init__(self, text_col='Descripcion'):
        self.text_col = text_col
        try:
            word_tokenize("Prueba")
        except:
            import nltk
            nltk.download('punkt')
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df_copy = X.copy()
        
        df_copy['words'] = df_copy[self.text_col].apply(self._tokenize_text)
        
        return df_copy
    
    def _tokenize_text(self, text):
        if not isinstance(text, str):
            return []
        
        # Tokenizar el texto
        tokens = word_tokenize(text)
        return tokens

In [6]:
class TokenPreprocessor(BaseEstimator, TransformerMixin):
    """
    Transformador personalizado para preprocesar tokens ya existentes
    """
    def __init__(self):
        try:
            self.stopwords = set(stopwords.words('spanish'))
        except:
            import nltk
            nltk.download('stopwords')
            self.stopwords = set(stopwords.words('spanish'))
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Crear una copia del DataFrame para no modificar el original
        df_copy = X.copy()
        
        # Aplicar el preprocesamiento a la columna 'words'
        df_copy['words'] = df_copy['words'].apply(self._preprocess_tokens)
        
        return df_copy
    
    def _preprocess_tokens(self, tokens):
        """Preprocesa una lista de tokens"""
        if not isinstance(tokens, list):
            return []
        
        # Convertir a minúsculas
        tokens = self._to_lowercase(tokens)
        
        # Eliminar puntuación
        tokens = self._remove_punctuation(tokens)
        
        # Eliminar caracteres no ASCII
        tokens = self._remove_non_ascii(tokens)
        
        # Eliminar stopwords
        tokens = self._remove_stopwords(tokens)
        
        return tokens
    
    def _to_lowercase(self, words):
        """Convertir a minúsculas"""
        return [word.lower() for word in words if word is not None]
    
    def _remove_punctuation(self, words):
        """Eliminar puntuación"""
        new_words = []
        for word in words:
            if word is not None:
                new_word = re.sub(r'[^\w\s]', '', word)
                if new_word != '':
                    new_words.append(new_word)
        return new_words
    
    def _remove_non_ascii(self, words):
        """Eliminar caracteres no ASCII"""
        new_words = []
        for word in words:
            if word is not None:
                new_word = unidecode.unidecode(word)
                new_words.append(new_word)
        return new_words
    
    def _remove_stopwords(self, words):
        """Eliminar stopwords"""
        new_words = []
        for word in words:
            if word not in self.stopwords:
                new_words.append(word)
        return new_words

In [7]:
class DuplicateHandler(BaseEstimator, TransformerMixin):
    """
    Transformador que maneja duplicados para entrenamiento y es seguro para predicción
    """
    def __init__(self, words_col='words', label_col='Label'):
        self.words_col = words_col
        self.label_col = label_col
        self.keep_indices_ = None  # Almacenará los índices a mantener
        self.is_fitted_ = False    # Para saber si estamos en modo entrenamiento o predicción
        
    def fit(self, X, y=None):
        # Trabajar con una copia para no modificar el original
        df = X.copy()
        
        # Verificar si estamos en modo predicción (sin Label)
        if self.label_col not in df.columns:
            self.is_fitted_ = True
            return self
            
        # Encontrar duplicados en la columna de palabras
        duplicados = df[df[self.words_col].duplicated(keep=False)]
        
        # Convertir las listas a tuplas para poder usarlas como índices en groupby
        duplicados_tuplas = duplicados[self.words_col].apply(tuple)
        
        # Encontrar los grupos donde hay conflicto en la etiqueta
        grupos = duplicados.groupby(duplicados_tuplas)
        conflictos = grupos[self.label_col].nunique() > 1
        
        # Obtener los índices de los duplicados conflictivos
        indices_conflicto = conflictos[conflictos].index if len(conflictos) > 0 else []
        filas_conflicto = duplicados[duplicados_tuplas.isin(indices_conflicto)].index if len(indices_conflicto) > 0 else []
        
        # Identificar las filas a mantener
        df_temp = df.drop(index=filas_conflicto)
        df_limpio = df_temp.drop_duplicates(subset=[self.words_col], keep="first")
        
        # Guardar los índices de las filas que queremos mantener
        self.keep_indices_ = df_limpio.index
        
        print(f"Filas originales: {len(X)}")
        print(f"Filas eliminadas por conflictos: {len(filas_conflicto)}")
        print(f"Filas eliminadas por duplicados simples: {len(df_temp) - len(df_limpio)}")
        print(f"Filas después de limpieza: {len(df_limpio)}")
        
        self.is_fitted_ = True
        return self
    
    def transform(self, X):
        # Si no estamos ajustados o no hay columna de palabras, devolver sin cambios
        if not self.is_fitted_ or self.words_col not in X.columns:
            return X
            
        # Si estamos en modo predicción (sin Label) o primer uso
        if self.label_col not in X.columns:
            # En predicción solo eliminamos duplicados, no buscamos conflictos
            df_copy = X.copy()
            return df_copy.drop_duplicates(subset=[self.words_col], keep="first")
            
        # Si estamos en modo entrenamiento y tenemos los índices
        if self.keep_indices_ is not None:
            # Filtrar solo las filas que queríamos mantener
            return X.loc[self.keep_indices_]
            
        return X

In [8]:
class Lemmatizer(BaseEstimator, TransformerMixin):
    """
    Transformador personalizado para realizar la lematización de verbos
    usando spaCy para una columna de tokens
    """
    def __init__(self):
        # Asegurarnos de que el modelo de spaCy está cargado
        try:
            self.nlp = spacy.load("es_core_news_sm")
        except:
            import sys
            !python -m spacy download es_core_news_sm
            self.nlp = spacy.load("es_core_news_sm")
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Crear copia del DataFrame
        df_copy = X.copy()
        
        # Aplicar lematización a la columna 'words'
        df_copy['words'] = df_copy['words'].apply(self._lemmatize_tokens)
        
        return df_copy
    
    def _lemmatize_tokens(self, tokens):
        """Lematiza una lista de tokens"""
        if not isinstance(tokens, list):
            return []
        
        # Unir los tokens en una cadena para procesarlos con spaCy
        text = " ".join(tokens)
        
        # Procesar el texto con spaCy
        doc = self.nlp(text)
        
        # Lematizar, enfocándose en los verbos
        lemmatized_tokens = [token.lemma_ if token.pos_ == "VERB" else token.text 
                            for token in doc]
        
        return lemmatized_tokens

In [9]:
class JoinTokens(BaseEstimator, TransformerMixin):
    """
    Transformador para unir tokens en una cadena de texto
    para preparar los datos para la vectorización
    """
    def __init__(self, words_col='words'):
        self.words_col = words_col
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Obtenemos solo la columna de tokens
        if isinstance(X, pd.DataFrame):
            # Si es un DataFrame, extraemos solo la columna de tokens
            tokens_lists = X[self.words_col]
        else:
            # Si ya es una serie o lista
            tokens_lists = X
            
        # Unimos los tokens en cadenas de texto
        return [' '.join(tokens) if isinstance(tokens, list) else '' for tokens in tokens_lists]

In [10]:
def create_text_classification_pipeline():
    pipeline = Pipeline([
        # 1. Tokenizar texto
        ('tokenizer', TextTokenizer(text_col='Descripcion')),
        
        # 2. Preprocesar tokens
        ('token_preprocessor', TokenPreprocessor()),
        
        # 3. Lematizar tokens
        ('lemmatizer', Lemmatizer()),
        
        # 4. Unir tokens para vectorización
        ('join_tokens', JoinTokens()),
        
        # 5. Vectorizar texto
        ('vectorizer', TfidfVectorizer()),
        
        # 6. Clasificar
        ('classifier', XGBClassifier(eval_metric='mlogloss',
                                   n_estimators=100,
                                   max_depth=5,
                                   learning_rate=0.1,
                                   random_state=42))
    ])
    
    return pipeline

In [11]:
def train_and_evaluate(df, text_col='Descripcion', target_col='Label', test_size=0.2, random_state=42):
    # Eliminar duplicados y conflictos a nivel de dataframe
    print(f"Filas originales: {len(df)}")
    
    # Detectar duplicados 
    duplicated_descriptions = df[df[text_col].duplicated(keep=False)]
    
    # Identificar duplicados conflictivos (mismo texto, diferente etiqueta)
    conflict_groups = duplicated_descriptions.groupby(text_col)[target_col].nunique() > 1
    conflict_texts = conflict_groups[conflict_groups].index
    
    # Eliminar filas con conflictos
    conflict_rows = df[df[text_col].isin(conflict_texts)].index
    df_clean = df.drop(index=conflict_rows)
    print(f"Filas eliminadas por conflictos: {len(conflict_rows)}")
    
    # Eliminar duplicados restantes
    df_clean = df_clean.drop_duplicates(subset=[text_col], keep='first')
    print(f"Filas después de eliminar duplicados y conflictos: {len(df_clean)}")
    
    # Dividir en entrenamiento y prueba
    df_train, df_test = train_test_split(
        df_clean, test_size=test_size, random_state=random_state, stratify=df_clean[target_col]
    )
    
    # Preparar X e y para entrenamiento
    X_train = df_train[[text_col]].copy()
    y_train = df_train[target_col]
    
    # Preparar X e y para prueba
    X_test = df_test[[text_col]].copy()
    y_test = df_test[target_col]
    
    # Crear y entrenar el pipeline
    pipeline = create_text_classification_pipeline()
    pipeline.fit(X_train, y_train)
    
    # Evaluar el modelo
    y_pred = pipeline.predict(X_test)
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    report = classification_report(y_test, y_pred)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")
    print("\nClassification Report:")
    print(report)
    
    return pipeline

In [12]:
def export_model(pipeline, file_path="modelo_clasificacion_texto.joblib"):
    joblib.dump(pipeline, file_path)
    print(f"Modelo exportado exitosamente a: {file_path}")
    return file_path

In [13]:
df = pd.read_csv("data/fake_news_spanish.csv", sep=";", encoding="utf-8")

In [14]:
df.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


In [15]:
# Ejecutar el pipeline de entrenamiento y evaluación
pipeline = train_and_evaluate(df, text_col='Descripcion', target_col='Label')

Filas originales: 57063
Filas eliminadas por conflictos: 13880
Filas después de eliminar duplicados y conflictos: 42787
Accuracy: 0.8662
F1 Score (weighted): 0.8602

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.68      0.80      3386
           1       0.82      0.99      0.90      5172

    accuracy                           0.87      8558
   macro avg       0.90      0.83      0.85      8558
weighted avg       0.88      0.87      0.86      8558



In [16]:
model_path = export_model(pipeline, "modelo_clasificacion_texto.joblib")

Modelo exportado exitosamente a: modelo_clasificacion_texto.joblib
